In [ ]:
# # File created for initial data cleaning and exploration

In [1]:
import pandas as pd
from prophet import Prophet
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Import Los Angeles crime data and moon files
LA_data_file = 'LA_Crime_Data_Source_File.csv'
moon_data_file = 'moon_data_export.csv'

# Read the data into a pandas dataframe
LA_df = pd.read_csv(LA_data_file)
moon_df = pd.read_csv(moon_data_file)

In [9]:
# Change Date column to datetime format
moon_df['Date'] = pd.to_datetime(moon_df['Date'])
LA_df['DATE OCC'] = pd.to_datetime(LA_df['DATE OCC'])

# Display the first 5 rows of the moon data
moon_df.info()
LA_df.info()

C:\Users\Sheila\AppData\Local\Temp\ipykernel_23592\1767251723.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  LA_df['DATE OCC'] = pd.to_datetime(LA_df['DATE OCC'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Moon Phase  500 non-null    object        
 1   Date        500 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 7.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004681 entries, 0 to 1004680
Data columns (total 28 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   DR_NO           1004681 non-null  int64         
 1   Date Rptd       1004681 non-null  object        
 2   DATE OCC        1004681 non-null  datetime64[ns]
 3   TIME OCC        1004681 non-null  int64         
 4   AREA            1004681 non-null  int64         
 5   AREA NAME       1004681 non-null  object        
 6   Rpt Dist No     1004681 non-null  int64         
 7   Part 1-2        1004681 non-null  int64  

In [10]:
# Check the first 5 rows of the LA crime data
LA_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,2020-03-01,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,2020-02-08,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,2020-11-04,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,2020-03-10,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,200412582,09/09/2020 12:00:00 AM,2020-09-09,630,4,Hollenbeck,413,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E AVENUE 28,NaN,34.0820,-118.2130


In [11]:
# Clean up LA crime data to only include relevant columns: Date as datetime, and Offense
# Fill in missing values and display the first 5 rows of the cleaned data

LA_crime_df = LA_df[['DATE OCC', 'Crm Cd Desc']]

# Rename columns
LA_crime_df.columns = ['Date', 'Offense']

# Convert 'Occurred Date' to datetime format and ensure consistency (set day as smallest increment)
LA_crime_df['Date'] = pd.to_datetime(LA_crime_df['Date']).dt.floor('D')

# Check for cleaned Austin data
LA_crime_df.info()
LA_crime_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004681 entries, 0 to 1004680
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   Date     1004681 non-null  datetime64[ns]
 1   Offense  1004681 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 15.3+ MB


C:\Users\Sheila\AppData\Local\Temp\ipykernel_23592\2230349433.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LA_crime_df['Date'] = pd.to_datetime(LA_crime_df['Date']).dt.floor('D')


,Date,Offense
0,2020-03-01,VEHICLE - STOLEN
1,2020-02-08,BURGLARY FROM VEHICLE
2,2020-11-04,BIKE - STOLEN
3,2020-03-10,SHOPLIFTING-GRAND THEFT ($950.01 & OVER)
4,2020-09-09,VEHICLE - STOLEN


In [12]:
moon_limited = moon_df.drop_duplicates(subset=["Date"])
moon_limited.shape

(495, 2)

In [13]:
# Merge dataframes and fill absent moon phase values with "Other"
merged_LA_df = LA_crime_df.merge(
    moon_limited,
    how="left",
    on="Date"
).fillna("Other")

#Display the first 5 rows of the merged data
merged_LA_df.info()
merged_LA_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004681 entries, 0 to 1004680
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   Date        1004681 non-null  datetime64[ns]
 1   Offense     1004681 non-null  object        
 2   Moon Phase  1004681 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 23.0+ MB


,Date,Offense,Moon Phase
0,2020-03-01,VEHICLE - STOLEN,Other
1,2020-02-08,BURGLARY FROM VEHICLE,Other
2,2020-11-04,BIKE - STOLEN,Other
3,2020-03-10,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),Other
4,2020-09-09,VEHICLE - STOLEN,Other


In [15]:
# Create a dictionary of full moon and other counts for total rows (i.e. individual crimes)
moon_phase_crimes = merged_LA_df["Moon Phase"].value_counts().to_dict()

# Replace specific moon phases with "Other"
moon_phase_crimes["Other"] = moon_phase_crimes.pop("New Moon", 0) + moon_phase_crimes.pop("First Quarter", 0) + moon_phase_crimes.pop("Last Quarter", 0)
moon_phase_crimes 

{'Other': 90837, 'Full Moon': 29585}

In [16]:
# Get a total count for non-full-moon dates (i.e. days with "Other")
other_dates = merged_LA_df[merged_LA_df["Moon Phase"] == "Other"]
days_other_moon = other_dates["Date"].nunique()
days_other_moon

1626

In [17]:
# Get a total count for individual crimes in each condition
days_full_moon = 124
full_moon_crimes = moon_phase_crimes.get("Full Moon")
other_moon_crimes = moon_phase_crimes.get("Other")
full_moon_crimes, other_moon_crimes 


(29585, 90837)

In [19]:
# Do the math for crime rate and percentage increase
full_crime_rate = full_moon_crimes / days_full_moon
other_crime_rate = other_moon_crimes / days_other_moon
percent_increase = (full_crime_rate / other_crime_rate - 1) * 100

print(f"The Los Angeles crime rate during a full moon is {full_crime_rate:.2f} against a control rate of {other_crime_rate:.2f}.")
print(f"These initial findings indicate a increased crime rate of {percent_increase:.2f}% above the base line.")

The Los Angeles crime rate during a full moon is 238.59 against a control rate of 55.87.
These initial findings indicate a increased crime rate of 327.08% above the base line.
